# Register a model using an AutoML experiment previously executed
In this notebook we use an experiment (`experiment_name` variable) previously executed to get and register the **Best Model**

In [35]:
experiment_name = 'experiment-diabetes'
model_name = 'diabetes-automl'

## Import Libs

In [36]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig

import warnings
warnings.filterwarnings('ignore')

In [37]:
print("This notebook was created using version 1.7.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was created using version 1.7.0 of the Azure ML SDK
You are currently using version 1.9.0 of the Azure ML SDK


## Get the Workspace and set `experiment_name` and `model_name`

In [38]:
ws = Workspace.from_config()

Here we create a function `get_last_run` to get the last run from an experiment

In [39]:
def get_last_run(ws, experiment_name):
    experiment = Experiment(ws, experiment_name)

    for run in experiment.get_runs(type='automl'):
        if(run.get_status()=='Completed'):
            return run

last_run = get_last_run(ws, experiment_name)

In [40]:
last_run

Experiment,Id,Type,Status,Details Page,Docs Page
experimentos-diabetes,AutoML_ed564fe5-9e90-4655-a1d2-4fb3240c417a,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


From this `last_run` we get the output to be able to use the `best_run`

In [41]:
best_run, fitted_model = last_run.get_output()

best_model_name = best_run.get_details()['properties']['model_name']
best_run_algorithm = best_run.get_details()['properties']['run_algorithm']
best_interaction = best_run.get_details()['properties']['iteration']

print('Model: {} trained using algorithm: {} in the interaction {}'.format(best_model_name,
                                                                           best_run_algorithm,
                                                                           best_interaction))

WARNING - Received unrecognized parameter enable_distributed_featurization
WARNING - Received unrecognized parameter enable_distributed_featurization


Model: AutoMLed564fe5958 trained using algorithm: VotingEnsemble in the interaction 58


`best_run` returns a `dict` with all the log of the **best model** trained. We got `best_model_name`, `best_run_algorithm` and the `best_interaction` but feel free to get more details about the **best_run**

## Now we can use the best_run to register the model to the Workspace
The outputs from the best_run can be found in the `outputs` folder and to register we just use the `model.pkl` from this folder.

In [52]:
best_run.register_model(model_name=model_name, model_framework='automl', model_path='outputs/model.pkl')

Model(workspace=Workspace.create(name='wk-qa', subscription_id='f56912be-98e5-44e3-9e64-54bc52cef4a7', resource_group='RG-Auto-ML'), name=diabetes-automl, id=diabetes-automl:30, version=30, tags={}, properties={})

In [16]:
from azureml.widgets import RunDetails
RunDetails(last_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…